# 🌳 Árboles de Decisión: Comparación de Profundidades
## Dataset: MNIST (Dígitos escritos a mano)

### Introducción Teórica

**¿Qué es un Árbol de Decisión?**

Un árbol de decisión es un modelo de aprendizaje supervisado que toma decisiones dividiendo el espacio de datos mediante preguntas binarias:

> ¿La característica X es mayor o menor que un valor?

**Conceptos clave:**

- **Profundidad (max_depth)**: Número máximo de niveles del árbol
- **Underfitting (Subajuste)**: Modelo demasiado simple, no captura patrones
- **Overfitting (Sobreajuste)**: Modelo demasiado complejo, memoriza ruido
- **Balance**: Profundidad óptima que generaliza bien

**Objetivo:** Comparar 3 árboles con diferentes profundidades para encontrar el balance óptimo.

## 1️⃣ Importar librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✓ Librerías importadas correctamente")

## 2️⃣ Carga y exploración del dataset MNIST

In [ ]:
print("="*60)
print("PASO 2: CARGA Y EXPLORACIÓN DEL DATASET")
print("="*60)

# Cargar datasets
train_df = pd.read_csv('../../datasets/mnist/mnist_train.csv')
test_df = pd.read_csv('../../datasets/mnist/mnist_test.csv')

print(f"\n✓ Dataset de entrenamiento cargado: {train_df.shape}")
print(f"✓ Dataset de prueba cargado: {test_df.shape}")

# Exploración
print(f"\nInformación del dataset de entrenamiento:")
print(f"  - Filas: {train_df.shape[0]:,}")
print(f"  - Columnas: {train_df.shape[1]}")
print(f"  - Valores nulos: {train_df.isnull().sum().sum()}")

print(f"\nPrimeras 5 filas:")
display(train_df.head())

print(f"\nEstadísticas descriptivas:")
display(train_df.describe())

# Descripción del dataset
print("\n" + "="*60)
print("DESCRIPCIÓN DEL DATASET")
print("="*60)
print("\n📝 ¿Qué representa cada fila?")
print("   Cada fila es una imagen de 28x28 píxeles de un dígito escrito a mano (0-9).")
print("   La imagen está 'aplanada' (flattened) en un vector de 784 valores.")
print("\n📝 ¿Qué contienen las columnas?")
print("   - Columna 0 (label): El dígito real (0-9) - VARIABLE OBJETIVO")
print("   - Columnas 1-784 (pixel0-pixel783): Intensidad de cada píxel (0-255)")
print("   - Cada píxel representa un punto en la imagen de 28x28")

## 3️⃣ Preparación de las variables (X y y)

In [ ]:
print("="*60)
print("PASO 3: PREPARACIÓN DE VARIABLES")
print("="*60)

# Separar X (features) y y (target)
# Asumiendo que la primera columna es 'label'
X_train_full = train_df.iloc[:, 1:].values  # Todas las columnas excepto la primera
y_train_full = train_df.iloc[:, 0].values   # Primera columna (label)

X_test_full = test_df.iloc[:, 1:].values
y_test_full = test_df.iloc[:, 0].values

print(f"\n✓ Variables preparadas:")
print(f"  - X_train shape: {X_train_full.shape}")
print(f"  - y_train shape: {y_train_full.shape}")
print(f"  - X_test shape: {X_test_full.shape}")
print(f"  - y_test shape: {y_test_full.shape}")

# Clases únicas
clases_unicas = np.unique(y_train_full)
print(f"\n✓ Clases únicas en y: {clases_unicas}")
print(f"✓ Número de clases: {len(clases_unicas)}")

# Distribución de clases
print(f"\nDistribución de clases en entrenamiento:")
for clase in clases_unicas:
    count = np.sum(y_train_full == clase)
    pct = (count / len(y_train_full)) * 100
    print(f"  Dígito {clase}: {count:,} ejemplos ({pct:.1f}%)")

print("\n" + "="*60)
print("EXPLICACIÓN")
print("="*60)
print("\n📝 ¿Qué significa cada fila en X?")
print("   Cada fila es un vector de 784 valores (28x28 píxeles aplanados).")
print("   Cada valor representa la intensidad de gris de un píxel (0=blanco, 255=negro).")
print("\n📝 ¿Qué representa y?")
print("   y contiene las etiquetas: el dígito real que representa cada imagen (0-9).")
print("   Es la variable que queremos predecir.")

### Visualización de ejemplos

In [ ]:
# Visualizar algunos dígitos
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
axes = axes.ravel()

for i in range(10):
    # Tomar un ejemplo de cada dígito
    idx = np.where(y_train_full == i)[0][0]
    imagen = X_train_full[idx].reshape(28, 28)
    
    axes[i].imshow(imagen, cmap='gray')
    axes[i].set_title(f'Dígito: {i}', fontsize=12, fontweight='bold')
    axes[i].axis('off')

plt.suptitle('Ejemplos de Dígitos MNIST', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('mnist_ejemplos.png', dpi=300, bbox_inches='tight')
print("\n✓ Gráfica guardada: mnist_ejemplos.png")
plt.show()

## 4️⃣ División en entrenamiento y prueba

In [ ]:
print("="*60)
print("PASO 4: DIVISIÓN TRAIN/TEST")
print("="*60)

# Ya tenemos train y test separados, pero vamos a usar una muestra más pequeña
# para que el entrenamiento sea más rápido (opcional)

# Usar una muestra del dataset para acelerar (puedes ajustar)
sample_size = 10000  # Usar 10,000 ejemplos de entrenamiento

# División 80/20 del dataset de entrenamiento
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full[:sample_size], 
    y_train_full[:sample_size],
    test_size=0.2,
    random_state=42,
    stratify=y_train_full[:sample_size]
)

print(f"\n✓ División completada:")
print(f"  - Entrenamiento: {X_train.shape[0]:,} ejemplos ({(len(X_train)/(len(X_train)+len(X_val)))*100:.0f}%)")
print(f"  - Validación: {X_val.shape[0]:,} ejemplos ({(len(X_val)/(len(X_train)+len(X_val)))*100:.0f}%)")
print(f"  - Test (separado): {X_test_full.shape[0]:,} ejemplos")

print(f"\n✓ Estratificación verificada:")
print(f"  Distribución en train:")
unique, counts = np.unique(y_train, return_counts=True)
for digit, count in zip(unique, counts):
    print(f"    Dígito {digit}: {count} ({count/len(y_train)*100:.1f}%)")

print(f"\n📝 Nota: Se usó random_state=42 para reproducibilidad.")
print(f"📝 Estratificación mantiene proporciones de cada dígito en train y val.")

## 5️⃣ Definir las profundidades a evaluar

In [ ]:
print("="*60)
print("PASO 5: DEFINICIÓN DE PROFUNDIDADES")
print("="*60)

# Profundidades a evaluar
profundidades = [5, 10, 20]

print(f"\n✓ Profundidades seleccionadas: {profundidades}")

print("\n" + "="*60)
print("JUSTIFICACIÓN")
print("="*60)
print("\n¿Por qué comparar varias profundidades?")
print("\n1. MNIST tiene 784 características (píxeles)")
print("   - Un árbol muy profundo puede memorizar el ruido (overfitting)")
print("   - Un árbol muy superficial puede no capturar patrones (underfitting)")
print("\n2. Profundidad 5 (BAJA):")
print("   - Árbol simple, pocas decisiones")
print("   - Puede subajustar (underfitting)")
print("   - Rápido de entrenar")
print("\n3. Profundidad 10 (MEDIA):")
print("   - Balance entre complejidad y generalización")
print("   - Probablemente el mejor punto")
print("\n4. Profundidad 20 (ALTA):")
print("   - Árbol muy complejo")
print("   - Puede sobreajustar (overfitting)")
print("   - Lento de entrenar")
print("\n✓ Comparar estas 3 nos permite encontrar el balance óptimo.")

## 6️⃣ Entrenamiento de los modelos (tres profundidades)

In [ ]:
print("="*60)
print("PASO 6: ENTRENAMIENTO DE MODELOS")
print("="*60)

# Diccionario para guardar modelos y resultados
modelos = {}
resultados = []

for profundidad in profundidades:
    print(f"\n{'='*60}")
    print(f"Entrenando árbol con profundidad = {profundidad}")
    print(f"{'='*60}")
    
    # Crear modelo
    modelo = DecisionTreeClassifier(
        max_depth=profundidad,
        random_state=42,
        criterion='gini'
    )
    
    # Entrenar
    print(f"Entrenando...")
    modelo.fit(X_train, y_train)
    print(f"✓ Entrenamiento completado")
    
    # Predicciones
    y_pred_train = modelo.predict(X_train)
    y_pred_val = modelo.predict(X_val)
    
    # Calcular accuracy
    acc_train = accuracy_score(y_train, y_pred_train)
    acc_val = accuracy_score(y_val, y_pred_val)
    
    # Guardar resultados
    modelos[profundidad] = modelo
    resultados.append({
        'Profundidad': profundidad,
        'Accuracy Train': acc_train,
        'Accuracy Val': acc_val,
        'Diferencia': acc_train - acc_val
    })
    
    print(f"\nResultados:")
    print(f"  - Accuracy Train: {acc_train:.4f} ({acc_train*100:.2f}%)")
    print(f"  - Accuracy Val:   {acc_val:.4f} ({acc_val*100:.2f}%)")
    print(f"  - Diferencia:     {acc_train - acc_val:.4f}")
    print(f"  - Nodos del árbol: {modelo.tree_.node_count}")
    print(f"  - Hojas del árbol: {modelo.get_n_leaves()}")

print(f"\n{'='*60}")
print(f"✓ Todos los modelos entrenados exitosamente")
print(f"{'='*60}")

## 7️⃣ Tabla de comparación

In [ ]:
print("="*60)
print("PASO 7: TABLA COMPARATIVA")
print("="*60)

# Crear DataFrame
df_resultados = pd.DataFrame(resultados)

print("\nTabla de Resultados:")
display(df_resultados.style.format({
    'Accuracy Train': '{:.4f}',
    'Accuracy Val': '{:.4f}',
    'Diferencia': '{:.4f}'
}).background_gradient(subset=['Accuracy Val'], cmap='RdYlGn'))

# Análisis
mejor_idx = df_resultados['Accuracy Val'].idxmax()
mejor_prof = df_resultados.loc[mejor_idx, 'Profundidad']
mejor_acc = df_resultados.loc[mejor_idx, 'Accuracy Val']

print("\n" + "="*60)
print("ANÁLISIS")
print("="*60)

print(f"\n✓ Mejor profundidad: {mejor_prof} (Accuracy Val = {mejor_acc:.4f})")

print("\n📊 Observaciones:")
for idx, row in df_resultados.iterrows():
    prof = row['Profundidad']
    diff = row['Diferencia']
    
    print(f"\nProfundidad {prof}:")
    if diff < 0.05:
        print(f"  ✓ Diferencia pequeña ({diff:.4f}) - Buen balance")
    elif diff < 0.15:
        print(f"  ⚠️ Diferencia moderada ({diff:.4f}) - Posible sobreajuste leve")
    else:
        print(f"  ✗ Diferencia grande ({diff:.4f}) - Sobreajuste evidente")

print("\n📝 Interpretación:")
print("  - Diferencia grande (train >> val) → Overfitting")
print("  - Accuracy bajo en ambos → Underfitting")
print("  - Diferencia pequeña y accuracy alto → Balance óptimo")

## 8️⃣ Gráfica de desempeño

In [ ]:
print("="*60)
print("PASO 8: GRÁFICA DE DESEMPEÑO")
print("="*60)

fig, ax = plt.subplots(figsize=(10, 6))

# Graficar líneas
ax.plot(df_resultados['Profundidad'], df_resultados['Accuracy Train'], 
        marker='o', linewidth=2.5, markersize=10, label='Train', color='blue')
ax.plot(df_resultados['Profundidad'], df_resultados['Accuracy Val'], 
        marker='s', linewidth=2.5, markersize=10, label='Validación', color='red')

# Configuración
ax.set_xlabel('Profundidad del Árbol', fontsize=12, fontweight='bold')
ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Desempeño de Árboles de Decisión vs Profundidad\nMNIST Dataset', 
            fontsize=14, fontweight='bold', pad=15)
ax.legend(fontsize=11, loc='best')
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xticks(df_resultados['Profundidad'])

# Añadir valores en los puntos
for idx, row in df_resultados.iterrows():
    ax.annotate(f"{row['Accuracy Train']:.3f}", 
               (row['Profundidad'], row['Accuracy Train']),
               textcoords="offset points", xytext=(0,10), ha='center', fontsize=9)
    ax.annotate(f"{row['Accuracy Val']:.3f}", 
               (row['Profundidad'], row['Accuracy Val']),
               textcoords="offset points", xytext=(0,-15), ha='center', fontsize=9)

plt.tight_layout()
plt.savefig('desempeno_profundidad.png', dpi=300, bbox_inches='tight')
print("\n✓ Gráfica guardada: desempeno_profundidad.png")
plt.show()

# Interpretación
print("\n" + "="*60)
print("INTERPRETACIÓN DE LA GRÁFICA")
print("="*60)

print("\n❓ ¿Aumentar profundidad siempre mejora el modelo?")
if df_resultados['Accuracy Val'].is_monotonic_increasing:
    print("   ✓ Sí, en este caso el accuracy de validación aumenta con la profundidad.")
else:
    print("   ✗ No, el accuracy de validación no siempre mejora.")
    print("   Aumentar profundidad puede causar overfitting.")

print("\n❓ ¿En qué punto comienza el sobreajuste?")
max_diff_idx = df_resultados['Diferencia'].idxmax()
prof_overfit = df_resultados.loc[max_diff_idx, 'Profundidad']
print(f"   La mayor diferencia train-val ocurre en profundidad {prof_overfit}.")
print(f"   Esto sugiere que el sobreajuste es más evidente ahí.")

print("\n❓ ¿Cuál profundidad logra el mejor balance?")
print(f"   Profundidad {mejor_prof} tiene el mejor accuracy de validación.")
mejor_diff = df_resultados.loc[mejor_idx, 'Diferencia']
print(f"   Con una diferencia train-val de {mejor_diff:.4f}.")
if mejor_diff < 0.1:
    print(f"   ✓ Excelente balance entre sesgo y varianza.")
else:
    print(f"   ⚠️ Hay algo de sobreajuste, pero es el mejor disponible.")